<div class="jumbotron">
    <center>
        <h1>The last mile</h1>
        <h2>Real Time Sentiment Analysis in 5 steps</h2>
    </center>
</div>

![](https://i.pinimg.com/564x/85/88/9a/85889a585974c7331b10343c98a53ee8.jpg)
[Pinterest](https://www.pinterest.ca/pin/255649716331177273/)

# 1. Get data from BlueSky 

:::: {.columns}

::: {.fragment .column width="50%"}
- Create App Password https://bsky.app/settings/app-passwords
- Write Scraper
- Setup .env (PYTHONUNBUFFERED=1 helps in stdout)
- Test ```docker compose run -e BLUESKY_LOGGER=stdout blueskyscraper```
::: 

::: {.fragment .column width="50%"}

:::
::::

# 2. Ingest data 

:::: {.columns}

::: {.fragment .column width="50%"}
- Create config for Fluentd (comment Kafka output)
- Test with stdout OK (limit docker compose up services)
- ```bash docker compose up blueskyscraper fluentd ```
- Test with Kafka
- Uncomment Kafka Output
- Start compose
- ```bash docker compose up blueskyscraper fluentd broker topics```
- Create consumer to test (profile debug?)
- ```docker exec broker /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server broker:9092 --topic bluetap --formatter kafka.tools.DefaultMessageFormatter --property print.key=true --property print.value=true --property key.deserializer=org.apache.kafka.common.serialization.StringDeserializer --property value.deserializer=org.apache.kafka.common.serialization.StringDeserializer  --from-beginning```
::: 

::: {.fragment .column width="50%"}

:::
::::


# 3. Process and Enrich Data
:::: {.columns}

::: {.fragment .column width="50%"}

- build tap:spark
- ```docker build spark --tag tap:spark```
- Please note that Spark is version 3.4.3 due to [ES issue](https://github.com/elastic/elasticsearch-hadoop/issues/2210)
- Delete
- ```rm -rf ./spark/code/models/sentitap/```
- train model
- ```docker run --hostname spark -p 4040:4040 -it --rm -v ./spark/code/:/opt/tap/ -v ./spark/dataset:/tmp/dataset tap:spark /opt/spark/bin/spark-submit /opt/tap/sentiment_analysis_model.py```
- Check model 
- ```ls -rtl ./spark/code/models/sentitap/```



::: 

::: {.fragment .column width="50%"}
- 
- Test Spark
- ```docker compose up blueskyscraper fluentd broker topics spark```
Notes
- added volume for ivy cache Done
- fix positive from 4 to 1
- 
:::
::::

# 4. Index Data
## from spark to elastic search

![](https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt7f9ca77d747fb53d/5c240aa34cca137b3874384e/es-hadoop-diagram.svg)

# Doc please
* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html

![](https://codinginfinite.com/wp-content/uploads/2019/05/documentation-where-were-going-we-dont-need-documentation.jpg)
[Source](https://codinginfinite.com/2019-programming-funny-jokes-memes/)

## 4.1 Define the index

Altough Elastic Search do automatically index creation and guess on field, 
sometimes is better to predefine the index and define mapping

This requires add elasticsearch python module to Spark docker image

```Dockerfile 
RUN pip install pyspark numpy elasticsearch
``` 

Then we need to create the mapping

#### Credit for mapping https://medium.com/@CMpoi/elasticsearch-defining-the-mapping-of-twitter-data-dafad0f50695 Timestamp
```python
es_mapping = {
    "mappings": {
        "properties": 
            {
                "created_at": {"type": "date","format": "EEE MMM dd HH:mm:ss Z yyyy"},
                "text": {"type": "text","fielddata": True}
            }
    }
}
```

Then connect to Elasticsearch and create the index

```python
es = Elasticsearch(hosts=elastic_host) 
# make an API call to the Elasticsearch cluster
# and have it return a response:
response = es.indices.create(
    index=elastic_index,
    body=es_mapping,
    ignore=400 # ignore 400 already exists code
)

if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])
```

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMTXhs9fd7M_qo4XA_6OiKLX9GaTldJWPICDtIGCB7SuE6q9NM&usqp=CAU)

## 4.2 Define Schema

```python
tweetKafka = tp.StructType([
    tp.StructField(name= 'id_str', dataType= tp.StringType(),  nullable= True),
    tp.StructField(name= 'created_at', dataType= tp.StringType(),  nullable= True),
    tp.StructField(name= 'text',       dataType= tp.StringType(),  nullable= True)
])
```

![](https://www.memecreator.org/static/images/memes/4772258.jpg)

## 4.3 Configure Elastic Search Parameters

```python
elastic_host="10.0.100.51"
elastic_index="tapwteet"
elastic_document="_doc"
}
```

![](http://www.quickmeme.com/img/6d/6d9b3966c39d711a732a842f41bc39f7023eb4fafdc9ecb22d0f8ca1c2fee05e.jpg)
[Source](http://www.quickmeme.com/meme/3tvnt7)

## 4.4 Take data

```python
   data2.select("id_str", "created_at", "text", "prediction") \
```

![](https://i.imgflip.com/3ddyn4.jpg)
[Source](https://imgflip.com/i/3ddyn4)

## 4.5 Send to ES

```python
 # Write the stream to elasticsearch
 df.writeStream \
    .option("checkpointLocation", "/save/location") \
    .format("es") \
    .start(elastic_index) \
 ```

```bash
# Start Elastic Search
bash elasticSearch.sh


bash sparkSubmitPythonV.sh mastodon_stream_elastic_nml.py "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.7.1"
```

![](https://i.chzbgr.com/full/2922317568/h5AC86B91/but-wait-theres-more)

# 5. Data Visualization
## Define Data View in  Kibana

# Biblio

* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html
* https://medium.com/@akkidx/indexing-into-elasticsearch-using-spark-code-snippets-55eabc753272
* https://github.com/vinayak0792/SparkStreaming/blob/master/consumer.py